In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

# Load the data
data = pd.read_csv('continuous dataset.csv')

# Convert timestamp to datetime format
data['datetime'] = data['datetime'][:45261].apply(lambda x: datetime.strptime(x, '%d-%m-%Y %H:%M'))

# Set the datetime as index
data.set_index('datetime', inplace=True)

# Resample the data to hourly frequency
data = data.resample('H').mean()

# Create lag features
for i in range(1, 25):
    data['lag_{}'.format(i)] = data['nat_demand'].shift(i)

# Create rolling mean and standard deviation features
data['rolling_mean'] = data['nat_demand'].rolling(window=24).mean()
data['rolling_std'] = data['nat_demand'].rolling(window=24).std()

# Create weekday and hour features
data['weekday'] = data.index.weekday
data['hour'] = data.index.hour

# Remove missing values
data.dropna(inplace=True)

# Split the data into train and test sets
train_size = int(len(data) * 0.8)
train_data = data[:train_size]/1000
test_data = data[train_size:]/1000

In [2]:
#Transformer
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, LayerNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error

# Prepare the train and test data
X_train, y_train = train_data.drop('nat_demand', axis=1), train_data['nat_demand']
X_test, y_test = test_data.drop('nat_demand', axis=1), test_data['nat_demand']

# Define the input shape
seq_len = X_train.shape[1]
n_features = 1

# Reshape the data for Transformer
X_train = X_train.values.reshape(X_train.shape[0], seq_len, n_features)
X_test = X_test.values.reshape(X_test.shape[0], seq_len, n_features)

# Define the Transformer model
inputs = Input(shape=(seq_len, n_features))
x = inputs

# Add self-attention layers
for _ in range(2):
    x = tf.keras.layers.MultiHeadAttention(num_heads=8, key_dim=32)(x, x)
    x = LayerNormalization(epsilon=1e-6)(x)
    x = Dropout(0.1)(x)
    x = tf.keras.layers.Dense(32, activation="relu")(x)

# Flatten and add output layer
x = tf.keras.layers.Flatten()(x)
outputs = Dense(1)(x)

model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(loss='mse', optimizer='adam')

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test), callbacks=[EarlyStopping(patience=5, restore_best_weights=True)])

# Evaluate the model
mae = history.history['val_loss'][-1]
mse = mean_squared_error(y_test, model.predict(X_test))
rmse = np.sqrt(mse)

ypred = model.predict(X_test)


Epoch 1/50
566/566 [==============================] - 67s 114ms/step - loss: 0.8838 - val_loss: 0.6048
Epoch 2/50
566/566 [==============================] - 64s 114ms/step - loss: 0.2793 - val_loss: 0.1975
Epoch 3/50
566/566 [==============================] - 62s 110ms/step - loss: 0.0819 - val_loss: 0.0749
Epoch 4/50
566/566 [==============================] - 62s 110ms/step - loss: 0.0412 - val_loss: 0.0482
Epoch 5/50
566/566 [==============================] - 67s 119ms/step - loss: 0.0369 - val_loss: 0.0435
Epoch 6/50
566/566 [==============================] - 74s 131ms/step - loss: 0.0367 - val_loss: 0.0429
Epoch 7/50
566/566 [==============================] - 67s 119ms/step - loss: 0.0367 - val_loss: 0.0430
Epoch 8/50
566/566 [==============================] - 61s 107ms/step - loss: 0.0367 - val_loss: 0.0433
Epoch 9/50
566/566 [==============================] - 61s 108ms/step - loss: 0.0367 - val_loss: 0.0430
Epoch 10/50
566/566 [==============================] - 62s 110ms/step - l

In [3]:
#For MAE
mae = np.mean(np.abs(ypred.ravel() - y_test.ravel()))
print('MAE = {:.2f} (%)'.format(mae))

#For MAPE
mape = np.mean(np.abs((y_test.ravel() - ypred.ravel()) / y_test.ravel())) * 100
print('MAPE = {:.2f} (%)'.format(mape))


# For RMSE
from sklearn.metrics import mean_squared_error 
mse=mean_squared_error(y_test,ypred) 
rsme=np.sqrt(mse) 
print('RSME = {:.2f} (%)'.format(rsme))

MAE = 0.17 (%)
MAPE = 12.95 (%)
RSME = 0.21 (%)
